# Lesson 7: Advanced stuff, decorators, \*args, \**kwargs, list comprehensions, generators, generator expressions and the itertools module.

# Chapters:
Chapter 10: Decorators (and args/kwargs) <br>
Chapter 11: Advanced Iterations (generators, comprehensions) <br>
Itertools module <br>
Author: Jurre Hageman <br>

## Decorators

Decorators are advanced topis but many modules and frameworks (like Flask) make use of them so a basic understanding of decorators is important. <br>
Decorators are functions that take another function and extend their behaviour without changing the code of the other function. Python supports the use of decorators with special syntactic sugar that simplifies their use. Let's start with some basic understanding of decorators:

Functions are objects in Python:

In [ ]:
def my_function():
    print("OK")

print(type(my_function))

And it is possible to pass a function as an argument in anouther function and invoke them within another function:

In [ ]:
def func1():
    print('two')


def func2(f):
    print('one')
    f()

    
func2(func1)

We can also define nested functions:

In [ ]:
def func1():
    print('one')
    def func2():
        print('two')
    func2()

func1()

However, due to the scoping rules we can not invoke func2 from the outer scope:

In [1]:
def func1():
    print('one')
    def func2():
        print('two')


func2()

NameError: name 'func2' is not defined

But we can also return a function from another function without invoking the second function:

In [2]:
def func1():
    print('one')
    def func2():
        print('two')
    return func2


x = func1()

one


The variable x now contains func2. We can invoke func2 as follows:

In [3]:
x()

two


If you understand the above concepts we can now continue to decorators. We first write a nested functions. The inner function is a wrapper function:

In [4]:
def decorate_function(function):
   def function_wrapper(name):
       return "DNA is composed of {}".format(function(name))
   return function_wrapper


def get_message(seq):
   return "the nucleotides {}".format(seq)


get_message = decorate_function(get_message)

print(get_message("ATCG"))

DNA is composed of the nucleotides ATCG


A lot is happening here. The 'decorate_function' contains an inner 'function_wrapper'. The 'decorate_function' also takes another function (get_message) as argument (this becomes a parameter in the function header). The inner function 'function_wrapper' invokes the function that 'decorate_function' received as argument and the 'wrapper_function' augments it's behaviour (it adds text to the string). Note that the 'function_wrapper' itself is not invoked. It is just returned by 'decorate_function'.

Let's now descibe the order of events that happens when the code runs:
Fist the two functions are declared:

In [11]:
def decorate_function(function):
   def function_wrapper(name):
       return "DNA is composed of {}".format(function(name))
   return function_wrapper


def get_message(seq):
   return "the nucleotides {}".format(seq)

Next, the following code is executed:

In [12]:
get_message = decorate_function(get_message)
print(get_message)

<function decorate_function.<locals>.function_wrapper at 0x1082390d0>


get_message is a variable that catches a function object... 'decorate_function' is invoked and the get_message function is used as an argument. Note that the get_message function is not invoked yet. The 'get_message' function is a parameter in 'decorate_function'. In 'decorate_function' is a nested function 'function_wrapper'. This function takes some text as argument and invokes 'get_message' when 'function_wrapper' get's invoked. But 'function_wrapper' is not invioked yet. It is returned as a function object. The 'get_message' function definition get's overwritten by the 'get_message' variable. So we can now invoke the 'function wrapper' with the original 'get_message' function by:

In [13]:
print(get_message("ATCG"))

DNA is composed of the nucleotides ATCG


Now the get_message function is decorated by the 'decorate_function'. Note that the behaviour of 'get_message' is changed but not it's code!

The above pattern is such an important pattern in Python that Python adds some syntactic sugar for it. We can rewrite the above as:

In [14]:
def decorate_function(function):
   def function_wrapper(name):
       return "DNA is composed of {}".format(function(name))
   return function_wrapper


@decorate_function
def get_message(seq):
   return "the nucleotides {}".format(seq)


print(get_message("ATCG"))

DNA is composed of the nucleotides ATCG


This reads as: 'add the functionality of the 'decorate_function' to get_message. And that is exactly what happened...

## \*args and \**kwargs

\*args and \**kwargs are also called magic variables. They can be very handy so it is important to understand them. Remember from the lessen about functions that a function call can contain arguments and the function header contains parameters: 

In [15]:
def my_func(param):
    print(param)
    
arg = "hello"
my_func(arg)

hello


However, if the number of arguments does not match the number of parameters an arror occurs:

In [16]:
def my_func(param):
    print(param)
    
arg1 = "hello"
arg2 = "world"
my_func(arg1, arg2)

TypeError: my_func() takes 1 positional argument but 2 were given

However, sometimes you might not know the number of arguments to expect. The *arg notation in the function header accepts any number of arguments. Only the * notation is important so you can also write *blablabla but do not do that as *args is used by convention:

In [18]:
def my_func(*args):
    print(args)
    
arg1 = "hello"
arg2 = "world"

my_func(arg1, arg2)
my_func()

('hello', 'world')
()


So now we do not have an error anymore. \*args in the function header accepts any number of positional arguments and is available in the function as a tuple. Invoking my_func without arguments results in an empty tuple. <br>
However, we can also use *args 